In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# === Config ===
INPUT_PATH = Path("Data_clean/Data_subject_complete.xlsx")  # phải có cột "split" với "train"/"val"/"test"
OUTPUT_SUMMARY = Path("Report/rf_training_summary_tvtt.xlsx")
RANDOM_STATE = 42

# === Load data ===
df = pd.read_excel(INPUT_PATH)
# bỏ ID nếu có
df_features = df.drop(columns=["EncryptedID"], errors="ignore")

# tách theo split
train_df = df_features[df_features["split"] == "train"].drop(columns=["split"])
val_df = df_features[df_features["split"] == "val"].drop(columns=["split"])
test_df = df_features[df_features["split"] == "test"].drop(columns=["split"])

summary_rows = []



# better explicit:
for target in [c for c in train_df.columns if c != "EncryptedID"]:
    print(f"\n=== RF Target: {target} ===")
    # features are all other columns except target
    features = [c for c in train_df.columns if c not in ("EncryptedID", target)]

    X_train = train_df[features]
    y_train = train_df[target]
    X_val = val_df[features]
    y_val = val_df[target]
    X_test = test_df[features]
    y_test = test_df[target]

    # GridSearch thủ công trên train, chọn theo val R2
    param_grid = {
        "n_estimators": [100, 200],
        "max_depth": [5, 10, None],
        "max_features": ["sqrt", 0.5],
        "min_samples_split": [2, 5],
        "min_samples_leaf": [1, 2]
    }

    best_score = np.inf  # nhỏ là tốt: RMSE
    best_params = None
    best_model = None

    for params in ParameterGrid(param_grid):
        model = RandomForestRegressor(random_state=RANDOM_STATE, **params)
        model.fit(X_train, y_train)
        pred_val = model.predict(X_val)
        rmse_val = np.sqrt(mean_squared_error(y_val, pred_val))
        if rmse_val < best_score:
            best_score = rmse_val
            best_params = params
            best_model = model


    # Đánh giá trên test
    y_pred_test = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred_test)
    mae = mean_absolute_error(y_test, y_pred_test)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred_test)

    print(f"Best params: {best_params} | Val R2={best_score:.4f} | Test R2={r2:.4f}")

    summary_rows.append({
        "Target": target,
        "Model": "RandomForest",
        "Best_Params": str(best_params),
        "Val_RMSE": best_score,
        "Test_MSE": mse,
        "Test_MAE": mae,
        "Test_RMSE": rmse,
        "Test_R2": r2
    })

# === Save summary ===
summary_df = pd.DataFrame(summary_rows)
OUTPUT_SUMMARY.parent.mkdir(parents=True, exist_ok=True)
summary_df.to_excel(OUTPUT_SUMMARY, index=False)
print(f"\n✅ RF train/val/test summary saved to: {OUTPUT_SUMMARY}")



=== RF Target: Giải tích II ===
Best params: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200} | Val R2=0.7218 | Test R2=0.1493

=== RF Target: Giải tích I ===
Best params: {'max_depth': None, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100} | Val R2=0.7617 | Test R2=0.1329

=== RF Target: Phương pháp tính ===
Best params: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100} | Val R2=0.7291 | Test R2=0.2419

=== RF Target: Đại số ===
Best params: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100} | Val R2=0.6879 | Test R2=0.1129

=== RF Target: Giải tích III ===
Best params: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100} | Val R2=0.6728 | Test R2=0.3922

=== RF Target: Xác suất thống kê ===
Best params: {'max_

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from xgboost import XGBRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# === Config ===
INPUT_PATH = Path("Data_clean/Data_subject_complete.xlsx")  # phải có cột "split" với values "train"/"val"/"test"
OUTPUT_SUMMARY = Path("Report/xgb_training_summary_tvtt.xlsx")
RANDOM_STATE = 42

# === Load data ===
df = pd.read_excel(INPUT_PATH)
# loại bỏ ID nếu có
df = df.drop(columns=["EncryptedID"], errors="ignore")

# tách theo split
train_df = df[df["split"] == "train"].drop(columns=["split"])
val_df = df[df["split"] == "val"].drop(columns=["split"])
test_df = df[df["split"] == "test"].drop(columns=["split"])

summary_rows = []

# === Loop từng target ===
for target in [c for c in train_df.columns if c != "EncryptedID"]:
    print(f"\n=== XGB Target: {target} ===")
    features = [c for c in train_df.columns if c not in ("EncryptedID", target)]

    X_train = train_df[features]
    y_train = train_df[target]
    X_val = val_df[features]
    y_val = val_df[target]
    X_test = test_df[features]
    y_test = test_df[target]

    # Grid search thủ công trên train, chọn theo val R2
    param_grid = {
        "n_estimators": [100, 200],
        "max_depth": [3, 6],
        "learning_rate": [0.05, 0.1],
        "subsample": [0.8, 1.0],
        "colsample_bytree": [0.8, 1.0],
        "reg_alpha": [0, 1],
        "reg_lambda": [1, 5]
    }

    best_score = np.inf  # tối thiểu hóa RMSE
    best_params = None
    best_model = None

    for params in ParameterGrid(param_grid):
        model = XGBRegressor(
            random_state=RANDOM_STATE,
            verbosity=0,
            objective="reg:squarederror",
            **params
        )
        model.fit(X_train, y_train)
        pred_val = model.predict(X_val)
        rmse_val = np.sqrt(mean_squared_error(y_val, pred_val))
        if rmse_val < best_score:
            best_score = rmse_val
            best_params = params
            best_model = model


    # Đánh giá trên test
    y_pred_test = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred_test)
    mae = mean_absolute_error(y_test, y_pred_test)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred_test)

    print(f"Best params: {best_params} | Val R2={best_score:.4f} | Test R2={r2:.4f}")

    summary_rows.append({
        "Target": target,
        "Model": "XGBoost",
        "Best_Params": str(best_params),
        "Val_RMSE": best_score,
        "Test_MSE": mse,
        "Test_MAE": mae,
        "Test_RMSE": rmse,
        "Test_R2": r2
    })

# === Save summary ===
summary_df = pd.DataFrame(summary_rows)
OUTPUT_SUMMARY.parent.mkdir(parents=True, exist_ok=True)
summary_df.to_excel(OUTPUT_SUMMARY, index=False)
print(f"\n✅ XGB train/val/test summary saved to: {OUTPUT_SUMMARY}")



=== XGB Target: Giải tích II ===
Best params: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 1, 'reg_lambda': 1, 'subsample': 0.8} | Val R2=0.7250 | Test R2=0.1381

=== XGB Target: Giải tích I ===
Best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 5, 'subsample': 0.8} | Val R2=0.7745 | Test R2=0.1348

=== XGB Target: Phương pháp tính ===
Best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 1, 'reg_lambda': 1, 'subsample': 0.8} | Val R2=0.7506 | Test R2=0.2089

=== XGB Target: Đại số ===
Best params: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 1.0} | Val R2=0.7153 | Test R2=0.0766

=== XGB Target: Giải tích III ===
Best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 100, 'r

In [3]:
import pandas as pd
import numpy as np
import ast
import random
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# === Cấu hình ===
DATA_PATH = Path("Data_clean/Data_subject_complete.xlsx")
RF_SUMMARY = Path("Report/rf_training_summary_tvtt.xlsx")  # phải chứa Best_Params + Val_RMSE
OUTPUT_PATH = Path("Report/rf_masking_training_summary.xlsx")
RANDOM_STATE = 42

# Số feature giả lập user nhập và số lần thử
feature_nums = [5, 10, 15, 20, 25]
trials_per = 10

# === Load dữ liệu ===
df = pd.read_excel(DATA_PATH)
df = df.drop(columns=["EncryptedID"], errors="ignore")

# Phân chia: nếu có cột "split" thì dùng, còn không thì chia 60/20/20

train_df = df[df["split"] == "train"].drop(columns=["split"])
val_df = df[df["split"] == "val"].drop(columns=["split"])
test_df = df[df["split"] == "test"].drop(columns=["split"])


# === Load best params từ summary RF ===
rf_summary = pd.read_excel(RF_SUMMARY)

# Hàm lấy params dict
def parse_params(s):
    try:
        return ast.literal_eval(s)
    except:
        return {}

# === Masking evaluation ===
results = []
subjects = [c for c in train_df.columns if c != "EncryptedID"]

for target in subjects:
    print(f"\n--- RF masking for target: {target} ---")
    # lấy best params; nếu không có thì skip
    row = rf_summary[rf_summary["Target"] == target]
    if row.empty:
        print(f"  No best params found for {target}, skipping.")
        continue
    best_params = parse_params(row.iloc[0]["Best_Params"])

    # toàn bộ feature còn lại
    all_features = [c for c in train_df.columns if c not in ("EncryptedID", target)]

    for num_feat in feature_nums:
        for trial in range(trials_per):
            selected = random.sample(all_features, num_feat)

            # Chuẩn bị X/y chỉ dùng các feature đã chọn
            X_train = train_df[selected]
            y_train = train_df[target]
            X_val = val_df[selected]
            y_val = val_df[target]
            X_test = test_df[selected]
            y_test = test_df[target]

            # Huấn luyện RF với best params (không cần imputer vì data sạch)
            model = RandomForestRegressor(random_state=RANDOM_STATE, **best_params)
            model.fit(X_train, y_train)

            # Dự đoán
            y_val_pred = model.predict(X_val)
            y_test_pred = model.predict(X_test)

            # Tính metrics
            rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
            r2_val = r2_score(y_val, y_val_pred)
            rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
            r2_test = r2_score(y_test, y_test_pred)
            mae_test = mean_absolute_error(y_test, y_test_pred)

            results.append({
                "Target": target,
                "Num_Features": num_feat,
                "Features_Used": ",".join(selected),
                "Val_RMSE": rmse_val,
                "Val_R2": r2_val,
                "Test_RMSE": rmse_test,
                "Test_R2": r2_test,
                "Test_MAE": mae_test,
            })

# === Lưu kết quả ===
out_df = pd.DataFrame(results)
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
out_df.to_excel(OUTPUT_PATH, index=False)
print(f"\n✅ Kết quả masking RF lưu tại: {OUTPUT_PATH}")



--- RF masking for target: Giải tích II ---

--- RF masking for target: Giải tích I ---

--- RF masking for target: Phương pháp tính ---

--- RF masking for target: Đại số ---

--- RF masking for target: Giải tích III ---

--- RF masking for target: Xác suất thống kê ---

--- RF masking for target: Vật lý đại cương II ---

--- RF masking for target: Vật lý đại cương I ---

--- RF masking for target: Tin học đại cương ---

--- RF masking for target: Vật lý điện tử ---

--- RF masking for target: Nhập môn kỹ thuật điện tử-viễn thông ---

--- RF masking for target: Thực tập cơ bản ---

--- RF masking for target: Technical Writing and Presentation ---

--- RF masking for target: Kỹ thuật lập trình C/C++ ---

--- RF masking for target: Cấu kiện điện tử ---

--- RF masking for target: Lý thuyết mạch ---

--- RF masking for target: Tín hiệu và hệ thống ---

--- RF masking for target: Lý thuyết thông tin ---

--- RF masking for target: Cơ sở kỹ thuật đo lường ---

--- RF masking for target: C

In [4]:
import pandas as pd
import numpy as np
import ast
import random
from pathlib import Path
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# === Cấu hình ===
DATA_PATH = Path("Data_clean/Data_subject_complete.xlsx")
XGB_SUMMARY = Path("Report/xgb_training_summary_tvtt.xlsx")  # phải chứa Best_Params + Val_RMSE (theo RMSE)
OUTPUT_PATH = Path("Report/xgb_masking_training_summary.xlsx")
RANDOM_STATE = 42

# Số feature giả lập user nhập và số lần thử
feature_nums = [5, 10, 15, 20, 25]
trials_per = 10

# === Load dữ liệu (đã có split) ===
df = pd.read_excel(DATA_PATH)
df = df.drop(columns=["EncryptedID"], errors="ignore")
train_df = df[df["split"] == "train"].drop(columns=["split"])
val_df = df[df["split"] == "val"].drop(columns=["split"])
test_df = df[df["split"] == "test"].drop(columns=["split"])

# === Load best params từ summary XGB ===
xgb_summary = pd.read_excel(XGB_SUMMARY)

def parse_params(s):
    try:
        return ast.literal_eval(s)
    except:
        return {}

# === Masking evaluation XGB ===
results = []
subjects = [c for c in train_df.columns if c != "EncryptedID"]

for target in subjects:
    print(f"\n--- XGB masking for target: {target} ---")
    row = xgb_summary[xgb_summary["Target"] == target]
    if row.empty:
        print(f"  No best params found for {target}, skipping.")
        continue
    best_params = parse_params(row.iloc[0]["Best_Params"])

    all_features = [c for c in train_df.columns if c not in ("EncryptedID", target)]

    for num_feat in feature_nums:
        for trial in range(trials_per):
            selected = random.sample(all_features, num_feat)

            X_train = train_df[selected]
            y_train = train_df[target]
            X_val = val_df[selected]
            y_val = val_df[target]
            X_test = test_df[selected]
            y_test = test_df[target]

            model = XGBRegressor(
                random_state=RANDOM_STATE,
                verbosity=0,
                objective="reg:squarederror",
                **best_params
            )
            model.fit(X_train, y_train)

            y_val_pred = model.predict(X_val)
            y_test_pred = model.predict(X_test)

            rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
            r2_val = r2_score(y_val, y_val_pred)
            rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
            r2_test = r2_score(y_test, y_test_pred)
            mae_test = mean_absolute_error(y_test, y_test_pred)

            results.append({
                "Target": target,
                "Num_Features": num_feat,
                "Features_Used": ",".join(selected),
                "Val_RMSE": rmse_val,
                "Val_R2": r2_val,
                "Test_RMSE": rmse_test,
                "Test_R2": r2_test,
                "Test_MAE": mae_test,
            })

# === Lưu kết quả ===
out_df = pd.DataFrame(results)
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
out_df.to_excel(OUTPUT_PATH, index=False)
print(f"\n✅ Kết quả masking XGB lưu tại: {OUTPUT_PATH}")



--- XGB masking for target: Giải tích II ---

--- XGB masking for target: Giải tích I ---

--- XGB masking for target: Phương pháp tính ---

--- XGB masking for target: Đại số ---

--- XGB masking for target: Giải tích III ---

--- XGB masking for target: Xác suất thống kê ---

--- XGB masking for target: Vật lý đại cương II ---

--- XGB masking for target: Vật lý đại cương I ---

--- XGB masking for target: Tin học đại cương ---

--- XGB masking for target: Vật lý điện tử ---

--- XGB masking for target: Nhập môn kỹ thuật điện tử-viễn thông ---

--- XGB masking for target: Thực tập cơ bản ---

--- XGB masking for target: Technical Writing and Presentation ---

--- XGB masking for target: Kỹ thuật lập trình C/C++ ---

--- XGB masking for target: Cấu kiện điện tử ---

--- XGB masking for target: Lý thuyết mạch ---

--- XGB masking for target: Tín hiệu và hệ thống ---

--- XGB masking for target: Lý thuyết thông tin ---

--- XGB masking for target: Cơ sở kỹ thuật đo lường ---

--- XGB m

In [5]:
import pandas as pd
import numpy as np
import ast
from pathlib import Path
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# === Config ===
DATA_PATH = Path("Data_clean/Data_subject_complete.xlsx")  # có cột "split"
RF_MASKING = Path("Report/rf_masking_training_summary.xlsx")  # chứa Feature_Used, Num_Features, và RF results
XGB_SUMMARY = Path("Report/xgb_training_summary_tvtt.xlsx")  # best params cho từng target
OUTPUT_COMPARE = Path("Report/xgb_vs_rf_masking_comparison.xlsx")
RANDOM_STATE = 42

# === Load data ===
df = pd.read_excel(DATA_PATH).drop(columns=["EncryptedID"], errors="ignore")
train_df = df[df["split"] == "train"].drop(columns=["split"])
val_df = df[df["split"] == "val"].drop(columns=["split"])
test_df = df[df["split"] == "test"].drop(columns=["split"])

# === Load RF masking combos and results ===
rf_masking_df = pd.read_excel(RF_MASKING)

# === Load best params for XGB per target ===
xgb_summary = pd.read_excel(XGB_SUMMARY)
def parse_params(s):
    try:
        return ast.literal_eval(s)
    except:
        return {}

# === Prepare results container ===
compare_rows = []

# === Loop từng target & each RF feature subset ===
for target in sorted(rf_masking_df["Target"].unique()):
    print(f"Processing target: {target}")
    # get best params for XGB target
    xgb_row = xgb_summary[xgb_summary["Target"] == target]
    if xgb_row.empty:
        print(f"  No XGB best params for {target}, skipping.")
        continue
    xgb_params = parse_params(xgb_row.iloc[0]["Best_Params"])

    # filter RF masking entries for this target (unique subsets)
    subset_df = rf_masking_df[rf_masking_df["Target"] == target]

    # Drop duplicates on Features_Used to avoid retraining same twice
    subset_df = subset_df.drop_duplicates(subset=["Features_Used", "Num_Features"])

    for _, row in subset_df.iterrows():
        feature_str = row["Features_Used"]
        selected_features = feature_str.split(",")
        num_feat = row["Num_Features"]

        # Build train/val/test splits only with selected features
        X_train = train_df[selected_features]
        y_train = train_df[target]
        X_val = val_df[selected_features]
        y_val = val_df[target]
        X_test = test_df[selected_features]
        y_test = test_df[target]

        # Train XGB with its best params
        model = XGBRegressor(
            random_state=RANDOM_STATE,
            verbosity=0,
            objective="reg:squarederror",
            **xgb_params
        )
        model.fit(X_train, y_train)

        # Predict
        y_val_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)

        # Metrics XGB
        val_rmse_xgb = np.sqrt(mean_squared_error(y_val, y_val_pred))
        val_r2_xgb = r2_score(y_val, y_val_pred)
        test_rmse_xgb = np.sqrt(mean_squared_error(y_test, y_test_pred))
        test_r2_xgb = r2_score(y_test, y_test_pred)
        test_mae_xgb = mean_absolute_error(y_test, y_test_pred)

        # Also extract the RF-side metrics from that row for comparison
        val_rmse_rf = row.get("Val_RMSE")
        val_r2_rf = row.get("Val_R2")
        test_rmse_rf = row.get("Test_RMSE")
        test_r2_rf = row.get("Test_R2")
        test_mae_rf = row.get("Test_MAE")

        compare_rows.append({
            "Target": target,
            "Num_Features": num_feat,
            "Features_Used": feature_str,
            # RF metrics
            "RF_Val_RMSE": val_rmse_rf,
            "RF_Val_R2": val_r2_rf,
            "RF_Test_RMSE": test_rmse_rf,
            "RF_Test_R2": test_r2_rf,
            "RF_Test_MAE": test_mae_rf,
            # XGB metrics
            "XGB_Val_RMSE": val_rmse_xgb,
            "XGB_Val_R2": val_r2_xgb,
            "XGB_Test_RMSE": test_rmse_xgb,
            "XGB_Test_R2": test_r2_xgb,
            "XGB_Test_MAE": test_mae_xgb,
        })

# === Save comparison ===
compare_df = pd.DataFrame(compare_rows)
OUTPUT_COMPARE.parent.mkdir(parents=True, exist_ok=True)
compare_df.to_excel(OUTPUT_COMPARE, index=False)
print(f"\n✅ So sánh RF vs XGB lưu tại: {OUTPUT_COMPARE}")


Processing target: Anten và truyền sóng
Processing target: Cơ sở kỹ thuật đo lường
Processing target: Cấu kiện điện tử
Processing target: Cấu trúc dữ liệu và giải thuật
Processing target: Giải tích I
Processing target: Giải tích II
Processing target: Giải tích III
Processing target: Kỹ thuật lập trình C/C++
Processing target: Kỹ thuật phần mềm ứng dụng
Processing target: Kỹ thuật vi xử lý
Processing target: Lý thuyết mạch
Processing target: Lý thuyết thông tin
Processing target: Nhập môn kỹ thuật điện tử-viễn thông
Processing target: Phương pháp tính
Processing target: Technical Writing and Presentation
Processing target: Thông tin số
Processing target: Thực tập cơ bản
Processing target: Tin học đại cương
Processing target: Trường điện từ
Processing target: Tín hiệu và hệ thống
Processing target: Vật lý điện tử
Processing target: Vật lý đại cương I
Processing target: Vật lý đại cương II
Processing target: Xác suất thống kê
Processing target: Xử lý tín hiệu số
Processing target: Điện tử

In [6]:
#Huấn luyện RF cuối cùng với best params từ summary, lưu model .joblib
import pandas as pd
import numpy as np
import ast
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib  # cần để lưu model
import re
# === Cấu hình ===
DATA_PATH = Path("Data_clean/Data_subject_complete.xlsx")  # phải có cột "split"
RF_SUMMARY = Path("Report/rf_training_summary_tvtt.xlsx")
OUTPUT_MODELS_DIR = Path("models_streamlit")  # nơi lưu .joblib
RANDOM_STATE = 42

# === Load dữ liệu ===
df = pd.read_excel(DATA_PATH)
df = df.drop(columns=["EncryptedID"], errors="ignore")

# Tách theo split
train_df = df[df["split"] == "train"].drop(columns=["split"])
val_df = df[df["split"] == "val"].drop(columns=["split"])
# Test không dùng để train lại, chỉ để đánh giá nếu cần
test_df = df[df["split"] == "test"].drop(columns=["split"])

# === Load summary params ===
rf_summary = pd.read_excel(RF_SUMMARY)

def parse_params(s):
    try:
        return ast.literal_eval(s)
    except:
        return {}

# Tạo thư mục lưu model
OUTPUT_MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Huấn luyện và lưu model cho từng target
for target in [c for c in train_df.columns if c != "EncryptedID"]:
    print(f"\n>>> Processing target: {target}")
    # Lấy params tốt nhất
    row = rf_summary[rf_summary["Target"] == target]
    if row.empty:
        print(f"  Không tìm được best params cho {target}, bỏ qua.")
        continue
    best_params = parse_params(row.iloc[0]["Best_Params"])

    features = [c for c in train_df.columns if c not in ("EncryptedID", target)]

    X_train = pd.concat([train_df[features], val_df[features]], axis=0)
    y_train = pd.concat([train_df[target], val_df[target]], axis=0)

    # Fit final model
    model = RandomForestRegressor(random_state=RANDOM_STATE, **best_params)
    model.fit(X_train, y_train)

    # (Tuỳ chọn) đánh giá nhanh trên test nếu muốn
    X_test = test_df[features]
    y_test = test_df[target]
    y_pred = model.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_test = r2_score(y_test, y_pred)
    print(f"  Test RMSE={rmse_test:.4f}, R2={r2_test:.4f}")

    # Lưu model
    safe_name = re.sub(r'[\\/:\"*?<>| ]+', "_", target).lower()
    out_path = OUTPUT_MODELS_DIR / f"rf_model_{safe_name}.joblib"
    joblib.dump(model, out_path)
    print(f"  ✅ Đã lưu model: {out_path}")



>>> Processing target: Giải tích II
  Test RMSE=0.7995, R2=0.1592
  ✅ Đã lưu model: models_streamlit\rf_model_giải_tích_ii.joblib

>>> Processing target: Giải tích I
  Test RMSE=0.7487, R2=0.1226
  ✅ Đã lưu model: models_streamlit\rf_model_giải_tích_i.joblib

>>> Processing target: Phương pháp tính
  Test RMSE=0.7692, R2=0.2534
  ✅ Đã lưu model: models_streamlit\rf_model_phương_pháp_tính.joblib

>>> Processing target: Đại số
  Test RMSE=0.7411, R2=0.1197
  ✅ Đã lưu model: models_streamlit\rf_model_đại_số.joblib

>>> Processing target: Giải tích III
  Test RMSE=0.6691, R2=0.3806
  ✅ Đã lưu model: models_streamlit\rf_model_giải_tích_iii.joblib

>>> Processing target: Xác suất thống kê
  Test RMSE=0.7400, R2=0.3417
  ✅ Đã lưu model: models_streamlit\rf_model_xác_suất_thống_kê.joblib

>>> Processing target: Vật lý đại cương II
  Test RMSE=0.6938, R2=0.3292
  ✅ Đã lưu model: models_streamlit\rf_model_vật_lý_đại_cương_ii.joblib

>>> Processing target: Vật lý đại cương I
  Test RMSE=0.7228, 